In [1]:
import requests
import json
from __future__ import print_function
import time
import locationiq
from locationiq.rest import ApiException
from pprint import pprint
import pandas
import time

In [3]:
data = pandas.read_csv('project-data-csv.csv')

In [4]:
data['osm_district'] = "" # Create placeholder for new column
data['dispute_district'] = "" # Create placeholder for new column

configuration = locationiq.Configuration()
# Configure API key authorization: key
configuration.api_key['key'] = 'YOUR_API_KEY'
# Uncomment below to setup prefix (e.g. Bearer) for API key, if needed
# configuration.api_key_prefix['key'] = 'Bearer'

# Defining host is optional and default to https://eu1.locationiq.com/v1
configuration.host = "https://eu1.locationiq.com/v1"
# Enter a context with an instance of the API client
with locationiq.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    api_instance = locationiq.ReverseApi(api_client)

format = 'json' # str | Format to geocode. Only JSON supported for SDKs
normalizecity = 1 # int | Normalizes village to city level data to city
addressdetails = 1 # int | Include a breakdown of the address into elements. Defaults to 1. (optional) (default to 1)
accept_language = 'en' # str | Preferred language order for showing search results, overrides the value specified in the Accept-Language HTTP header. Defaults to en. To use native language for the response when available, use accept-language=native (optional)
namedetails = 0 # int | Include a list of alternative names in the results. These may include language variants, references, operator and brand. (optional)
extratags = 0 # int | Include additional information in the result if available, e.g. wikipedia link, opening hours. (optional)
statecode = 0 # int | Adds state or province code when available to the statecode key inside the address element. Currently supported for addresses in the USA, Canada and Australia. Defaults to 0 (optional)
showdistance = 0 # int | Returns the straight line distance (meters) between the input location and the result's location. Value is set in the distance key of the response. Defaults to 0 [0,1] (optional)
postaladdress = 0 # int | Returns address inside the postaladdress key, that is specifically formatted for each country. Currently supported for addresses in Germany. Defaults to 0 [0,1] (optional)

for index, row in data.iterrows():
    # print(row['coordinates_obscured'])
    if row['coordinates_obscured'] == True:    # Obscured coordinates are not accurate, for obvious reasons
        state_district = row['place_admin2_name']
    elif:
        lat = row['latitude'] # float | Latitude of the location to generate an address for.
        lon = row['longitude'] # float | Longitude of the location to generate an address for.
        try:
            # Reverse Geocoding
            api_response = api_instance.reverse(lat, lon, format, normalizecity, addressdetails=addressdetails, accept_language=accept_language, namedetails=namedetails, extratags=extratags, statecode=statecode, showdistance=showdistance, postaladdress=postaladdress)
            # with open("log.txt", "a+") as logfile:
            #    pprint(api_response, logfile)
        except ApiException as e:
            print("Exception when calling ReverseApi->reverse: %s\n" % e)
        if api_response.address.state_district == None:
            state_district = api_response.address.village # OSM returns empty for district, in which case, village seem to be next reliable field
        else:
            state_district = api_response.address.state_district.split(" ", 1)[0] # OSM returns "district" suffix for some districts. 
    data.at[index, 'osm_district'] = state_district # Record district info from OSM
    if row['place_admin2_name'] == state_district:
        data.at[index, 'dispute_district'] = 'no' # iNat and OSM geocoding matches
    else:
        data.at[index, 'dispute_district'] = 'yes' # iNat and OSM geocoding mismatches
    time.sleep(1.005) # LocationIQ Free Tier Rate Limits: 2/sec, 60/min, 10000/day

In [5]:
#data[['place_admin2_name', 'osm_district', 'dispute_district']]
data.to_csv('output.csv')